# Basis Set Dependence
In this notebook we will check the basis set dependence of the purity indicator and bond orders.

In [ ]:
from os.path import join
geom = "renumber"
bsets = ["STO-3G", "MINI", "MIDI",
         "pcseg-0", "pcseg-1", "pcseg-2", "pcseg-3",
         "def2-SVP", "def2-SVPD", "def2-TZVP", "def2-TZVPD"]

markers = {}
msize = {}
markers["STO_3G"] = "+"
markers["MINI"] = "x"
markers["MIDI"] = "d"
msize["STO_3G"] = 12
msize["MINI"] = 12
msize["MIDI"] = 8

markers["pcseg_0"] = "+"
markers["pcseg_1"] = "x"
markers["pcseg_2"] = "d"
markers["pcseg_3"] = "v"
msize["pcseg_0"] = 12
msize["pcseg_1"] = 12
msize["pcseg_2"] = 8
msize["pcseg_3"] = 8

markers["def2_SVP"] = "+"
markers["def2_SVPD"] = "x"
markers["def2_TZVP"] = "d"
markers["def2_TZVPD"] = "v"
msize["def2_SVP"] = 12
msize["def2_SVPD"] = 12
msize["def2_TZVP"] = 8
msize["def2_TZVPD"] = 8

Read in the file.

In [ ]:
from pyntchem.io import read_pdb
with open(join("input", geom + ".pdb")) as ifile:
    sys = read_pdb(ifile)

Setup the input file.

In [ ]:
from pyntchem.basis import BasisSet
basis = [BasisSet(x, sys.get_symlookup()) for x in bsets]

In [ ]:
from pyntchem.inputfile import Inputfile
inp = Inputfile()
inp.set_basic_dft(xtype="pbe", ctype="pbe")
inp.set_dft_prune_grid(50, 194)
inp.set_scf_guess("gwh")
inp["scf"].pulayperiod = 2
inp["scf"].facdamp = 0.95
inp["scf"].exchtype = "None"
inp["scf"].writeanal = True

inp["int2"].prelinkjthreshold = 1e-8
inp["int2"].prelinkkthreshold = 1e-4

In [ ]:
from pyntchem.calculator import JobscriptCalculator
calc = JobscriptCalculator(computer="Spring", skip=True, verbose=True)
args = {"nodes": 1, "tasks_per_node": 4, "omp": 9,  "queue": "winter2"}

First calculation.

In [ ]:
calc.run(sys, inp, basis[0], name=basis[0].name, 
         run_dir=join("work-basis"),  **args)

In [ ]:
from time import sleep
while not calc.check_results(): sleep(10.0)
logfiles = {x: y.log for x, y in calc.calculations.items()}

Remaining basis sets.

In [ ]:
from pyntchem.preprocessing import put_guess_matrix
from contextlib import suppress

inp.set_scf_guess("readdens")
inp.set_project()
inp["scf"].vshift = 0
for i in range(1, len(basis)):
    alp = logfiles[basis[i-1].name].densalp
    with suppress(IOError):
        put_guess_matrix(join("work-basis"), 
                         basis[i].name, dens_alp_file=alp)
    
    calc.run(sys, inp, basis[i], name=basis[i].name, 
             basis_set_proj = basis[i-1],
             run_dir=join("work-basis"),  **args)
    
    while not calc.check_results(): sleep(10.0)
    logfiles = {x: y.log for x, y in calc.calculations.items()}

Post-process to look at the purity values.

In [ ]:
from pyntchem.postprocessing import NTChemTool
tool = NTChemTool()

purities = {}
for b in basis:
    log = logfiles[b.name]
    purities[b.name] = tool.run_compute_purity(sys, log)

Also bond order values.

In [ ]:
bond_orders = {}
for b in basis:
    log = logfiles[b.name]
    bond_orders[b.name] = tool.fragment_bond_order(sys, list(sys), list(sys), log)

Read in BigDFT results.

In [ ]:
from pickle import load
with open("bigdft.cache", "rb") as ifile:
    purities["BigDFT"], bond_orders["BigDFT"] = load(ifile)

Plot purity values.

In [ ]:
order = sorted(sys, key=lambda x: int(x.split(":")[1]))

In [ ]:
from matplotlib import pyplot as plt
fig, axs = plt.subplots(1, 3, figsize=(12, 5))

for b in basis:
    if "pcseg" in b.name:
        i = 1
        n = b.name.upper()
    elif "def2" in b.name:
        i = 2
        n = b.name
    else:
        i = 0
        n = b.name
    n = n.replace("_", "-")
    axs[i].plot([purities[b.name][k] for k in order], 
                label=n, marker=markers[b.name],
                markersize=msize[b.name])
    
for i in range(3):
    axs[i].plot([purities["BigDFT"][k] for k in order], 
                label="BigDFT", linestyle="--", color='k', linewidth=2)
    
for i in range(3):
    axs[i].legend(loc="lower center", ncol=2)
    axs[i].set_ylim(-0.13, 0)
    axs[i].set_xticks(range(len(order)))
    axs[i].set_xticklabels(order, rotation=90)
    axs[i].tick_params(axis='both', which='major', labelsize=16)

axs[0].set_title("Minimal", fontsize=18)
axs[1].set_title("Jensen", fontsize=18)
axs[2].set_title("Karlsruhe", fontsize=18)
axs[0].set_ylabel("Purity Indicator", fontsize=18)
fig.tight_layout()
fig.savefig("basis-pi.png", dpi=600)

Plot bond orders.

In [ ]:
relevant = []
for k1, v1 in bond_orders["BigDFT"].items():
    for k2, v2 in v1.items():
        if k1 == k2:
            continue
        if (k2, k1) in relevant:
            continue
        if "MOL" in k1:
            if v2 > 1e-2:
                relevant.append((k1, k2))
        else:
            if v2 > 5e-3:
                relevant.append((k1, k2))
relevant = sorted(relevant, key=lambda x: int(x[0].split(":")[1]))

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 5))

for b in basis:
    if "pcseg" in b.name:
        i = 1
        n = b.name.upper().replace("_", "-")
    elif "def2" in b.name:
        i = 2
        n = b.name
    else:
        i = 0
        n = b.name
    n = n.replace("_", "-")
    axs[i].plot([bond_orders[b.name][k[0]][k[1]] for k in relevant], 
                label=n, marker=markers[b.name],
                markersize=msize[b.name])
    
for i in range(3):
    axs[i].plot([bond_orders["BigDFT"][k[0]][k[1]] for k in relevant], 
                label="BigDFT", linestyle="--", color='k', linewidth=2)
    
for i in range(3):
    axs[i].legend(loc="lower center", ncol=2)
    axs[i].set_ylim(5e-4, 1)
    axs[i].set_xticks(range(len(relevant)))
    axs[i].set_xticklabels(["-".join(x) for x in relevant], rotation=90)
    axs[i].set_yscale("log")
    axs[i].tick_params(axis='both', which='major', labelsize=16)

axs[0].set_title("Minimal", fontsize=18)
axs[1].set_title("Jensen", fontsize=18)
axs[2].set_title("Karlsruhe", fontsize=18)
axs[0].set_ylabel("Fragment Bond Order", fontsize=18)
fig.tight_layout()
fig.savefig("basis-fbo.png", dpi=600)